🐍 第10回講義用 Pythonデモコード

（重み付き残差法・ガラーキン法の可視化）

このコードは、資料にあるような微分方程式（例：$-u'' = 2$ など）に対し、試行関数（近似関数）を使って近似解を求め、厳密解と比較するものです。

In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

def demo_galerkin_method():
    # 1. 記号の定義
    x, L = sp.symbols('x L')

    # --- 問題設定 ---
    # 解くべき微分方程式: -u''(x) - f(x) = 0
    # 境界条件: u(0) = 0, u(L) = 0
    f = 2  # 外力項（一定と仮定）
    L_val = 1.0 # Lの具体的な値（グラフ描画用）

    # 2. 近似関数（試行関数）の設定
    # 境界条件 u(0)=0, u(L)=0 を満たす関数として以下を仮定
    # u_approx(x) = c1 * x * (L - x)
    c1 = sp.Symbol('c1')
    u_approx = c1 * x * (L - x)

    print("--- 1. 近似関数の仮定 ---")
    print(f"u_approx(x) = {u_approx}")

    # 3. 重み付き残差法（ガラーキン法）による計算
    # 残差 R(x) = -u'' - f
    R = -sp.diff(u_approx, x, 2) - f

    # 重み関数 w(x) （ガラーキン法では試行関数の基底と同じ形を使う）
    w = x * (L - x)

    # 重み付き残差積分 ∫ R * w dx = 0 を解く
    integral_eq = sp.integrate(R * w, (x, 0, L))

    # c1について解く
    sol_c1 = sp.solve(integral_eq, c1)[0]

    print("\n--- 2. 係数 c1 の決定 ---")
    print(f"重み付き残差積分 = {integral_eq} = 0")
    print(f"計算された係数 c1 = {sol_c1}")

    # 最終的な近似解
    u_final = u_approx.subs(c1, sol_c1)
    print(f"近似解 u(x) = {u_final}")

    # 4. 厳密解の導出（比較用）
    # -u'' = 2  => u'' = -2 => u' = -2x + C => u = -x^2 + Cx + D
    # u(0)=0 => D=0, u(L)=0 => -L^2 + CL = 0 => C=L
    # 厳密解 u_exact = -x^2 + Lx
    u_exact = -x**2 + L * x
    print(f"厳密解 u_exact(x) = {u_exact}")

    # --- 5. 可視化 (Matplotlib) ---
    # 数式をPython関数化 (lambdify)
    func_approx = sp.lambdify(x, u_final.subs(L, L_val), 'numpy')
    func_exact = sp.lambdify(x, u_exact.subs(L, L_val), 'numpy')

    xs = np.linspace(0, L_val, 100)
    ys_approx = func_approx(xs)
    ys_exact = func_exact(xs) # この例では近似解と厳密解が一致するはず

    plt.figure(figsize=(8, 5))
    plt.plot(xs, ys_exact, 'k-', linewidth=3, label='Exact Solution', alpha=0.5)
    plt.plot(xs, ys_approx, 'r--', linewidth=2, label='Approximate (Galerkin)')
    plt.fill_between(xs, ys_exact, ys_approx, color='red', alpha=0.1)

    plt.title(f"Galerkin Method Approximation (L={L_val})")
    plt.xlabel("x")
    plt.ylabel("u(x)")
    plt.legend()
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    demo_galerkin_method()

🐍 Pythonデモコード：多項近似によるテーパー棒の解析
このコードでは、近似関数の項数（$N$）を増やすことで、「マトリックスサイズが大きくなり、解が真値に収束していく様子」を視覚的に示せます。これぞ「マトリックス構造解析」の真骨頂です。

In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

def demo_complex_galerkin():
    # 1. 変数と問題設定
    x = sp.Symbol('x')
    L_val = 1.0

    # --- 複雑な問題設定 (テーパー棒) ---
    # 断面積 A(x) が場所によって変わる（根元が2、先端が1）
    E = 1.0
    A = 2.0 - x  # 断面積 A(x) = 2 - x
    f = 10.0     # 分布荷重 (一定)

    # 支配方程式: -(EA u')' = f
    # 厳密解の導出 (比較用)
    # EA u' = -fx + C1 => u' = (-fx + C1)/EA
    # u = int( (-fx + C1)/(E(2-x)) ) dx ... 手計算だと面倒！

    # ここではPython(SymPy)に厳密解も解かせてしまいます
    u_exact_func = sp.Function('u')(x)
    diff_eq = -sp.diff(E * A * sp.diff(u_exact_func, x), x) - f
    # 境界条件: u(0)=0, u(L)=0 (両端固定と仮定)
    sol_exact = sp.dsolve(diff_eq, u_exact_func, ics={u_exact_func.subs(x, 0): 0, u_exact_func.subs(x, L_val): 0})
    u_exact_expr = sol_exact.rhs
    print(f"厳密解（対数を含む複雑な式）:\n u(x) = {u_exact_expr}\n")

    # 2. ガラーキン法による近似 (N項)
    # 試行関数 u = Σ c_i * φ_i(x)
    # φ_i(x) = x^i * (L - x)  (x=0, x=Lで0になる多項式)

    N_terms = [1, 2, 3] # 項数N=1, 2, 3 の場合を比較

    plt.figure(figsize=(10, 6))

    # 厳密解のプロット
    func_exact_np = sp.lambdify(x, u_exact_expr, 'numpy')
    xs = np.linspace(0, L_val, 100)
    plt.plot(xs, func_exact_np(xs), 'k-', linewidth=4, alpha=0.3, label='Exact Solution')

    # N項近似のループ
    for N in N_terms:
        print(f"--- 近似項数 N = {N} の場合 ---")

        # 基底関数 φ_i のリスト作成
        phis = [x**i * (L_val - x) for i in range(1, N+1)]

        # 剛性行列 [K] と 荷重ベクトル {F} の構築
        K_mat = np.zeros((N, N))
        F_vec = np.zeros(N)

        # 行列要素の積分計算
        for i in range(N):
            # 弱形式の右辺: ∫ f * φ_i dx
            F_vec[i] = sp.integrate(f * phis[i], (x, 0, L_val))

            for j in range(N):
                # 弱形式の左辺: ∫ EA * φ_i' * φ_j' dx
                # (部分積分により、微分の積になるのがポイント)
                integrand = E * A * sp.diff(phis[i], x) * sp.diff(phis[j], x)
                K_mat[i, j] = sp.integrate(integrand, (x, 0, L_val))

        # 連立方程式を解く [K]{c} = {F}
        c_coeffs = np.linalg.solve(K_mat, F_vec)

        print(f"  剛性行列 K ({N}x{N}):\n{K_mat}")
        print(f"  係数 c: {c_coeffs}")

        # 近似解の構成
        u_approx = sum(c * phi for c, phi in zip(c_coeffs, phis))

        # プロット
        func_approx_np = sp.lambdify(x, u_approx, 'numpy')
        plt.plot(xs, func_approx_np(xs), linewidth=2, linestyle='--', label=f'Approx (N={N})')

    plt.title(f"Finite Element Approximation (Tapered Bar: A=2-x)")
    plt.xlabel("x (m)")
    plt.ylabel("Displacement u(x)")
    plt.legend()
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    demo_complex_galerkin()

🐍 Pythonデモコード：2次元膜のたわみ解析（数千自由度）

このコードは、正方形の領域を細かいグリッド（要素）に分割し、圧力による変形を計算します。 グリッド数 N を変更するだけで、自由度を自在に変えられます。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
from scipy.sparse.linalg import spsolve
import time

def solve_large_dof_2d(N=50):
    """
    N: グリッドの分割数。
    自由度は (N+1) * (N+1) になります。
    例: N=100 なら 約1万自由度
    """
    print(f"--- 2次元ポアソン方程式の解析 (分割数 N={N}) ---")

    # 1. メッシュ作成
    L = 1.0
    x = np.linspace(0, L, N+1)
    y = np.linspace(0, L, N+1)
    X, Y = np.meshgrid(x, y)

    num_nodes = (N+1) * (N+1)
    h = L / N # グリッド間隔

    print(f"総自由度数: {num_nodes:,} DOFs")

    # 2. 剛性行列 [K] の作成（疎行列として構築）
    # 差分法/有限要素法(1次要素)によるラプラシアン行列の構築
    # -u_xx - u_yy = f

    # データの準備 (LIL形式は構築が高速)
    K = sparse.lil_matrix((num_nodes, num_nodes))
    F = np.zeros(num_nodes)

    # 境界条件のためのインデックス管理
    # グリッド (i, j) -> 全体インデックス k
    to_idx = lambda i, j: i * (N+1) + j

    print("行列構築中...", end="", flush=True)
    start_time = time.time()

    # 全節点をループ（※本来はベクトル化推奨ですが、教育用にループで書きます）
    for i in range(1, N):
        for j in range(1, N):
            k = to_idx(i, j)

            # 支配方程式（5点ステンシル）
            # (4u_{i,j} - u_{i-1,j} - u_{i+1,j} - u_{i,j-1} - u_{i,j+1}) / h^2 = f

            # 対角成分
            K[k, k] = 4.0 / h**2

            # 隣接成分
            neighbors = [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]
            for ni, nj in neighbors:
                nk = to_idx(ni, nj)
                K[k, nk] = -1.0 / h**2

            # 荷重項 (等分布荷重 f=1.0)
            F[k] = 1.0

    # 境界条件処理 (Dirichlet: 周囲を0固定)
    # 行列構築時に i=0, N, j=0, N の部分はループを飛ばしているので
    # デフォルトで 0 (つまりK[k,k]=1, F[k]=0 とすれば u=0になる)
    # ただしlil_matrixの初期値は0なので、対角成分に1を入れて正則化する
    for i in range(N+1):
        for j in range(N+1):
            if i==0 or i==N or j==0 or j==N:
                k = to_idx(i, j)
                K[k, k] = 1.0
                F[k] = 0.0

    # 疎行列形式(CSR)に変換して計算効率化
    K_csr = K.tocsr()
    print(f"完了 ({time.time() - start_time:.2f}秒)")

    # 3. 連立方程式を解く (疎行列ソルバー使用)
    print("連立方程式を解いています...", end="", flush=True)
    start_time = time.time()
    U_vec = spsolve(K_csr, F)
    print(f"完了 ({time.time() - start_time:.2f}秒)")

    # 4. 結果の可視化
    U_mat = U_vec.reshape((N+1, N+1))

    plt.figure(figsize=(10, 8))

    # コンター図 (2D)
    plt.subplot(2, 1, 1)
    plt.contourf(X, Y, U_mat, 50, cmap='jet')
    plt.colorbar(label='Displacement u')
    plt.title(f"2D Poisson Analysis (N={N}, Nodes={num_nodes:,})")
    plt.xlabel('x'); plt.ylabel('y')
    plt.axis('equal')

    # 3D曲面図
    ax = plt.subplot(2, 1, 2, projection='3d')
    surf = ax.plot_surface(X, Y, U_mat, cmap='jet', edgecolor='none', alpha=0.9)
    ax.set_title('3D Deformation')
    ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_zlabel('u')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    # Nの値を変更して試してみてください
    # N=20 -> 441自由度 (一瞬)
    # N=100 -> 10,201自由度 (数秒)
    # N=200 -> 40,401自由度 (PCによっては少し重い)
    solve_large_dof_2d(N=5)